## Importing Dependencies

In [1]:
import requests
from htmldate import find_date
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup
import ssl
from tqdm import tqdm
from textblob import TextBlob
import openai

from html_extractor import *
from get_suburls import *
from openai_func import *
from get_date import *

from keyword_extraction import keyword_extractor_paragraph as kep

## Extracting sub urls

In [4]:
urls_list = ["https://www.khaleejtimes.com" , "https://www.indiatoday.in"]
urls_list_str = ",".join(urls_list)

keywords = "gaza,israel,hamas,idf"

scraper = WebScraper2(sub_url_size = 3 , keywords = keywords)
                        # Integration with DB will make it faster in future, as fetching is much faster than scrapping.
inside_urls, failed_fetch, sub_url_size, total_size = scraper.get_suburls2(urls_list_str)

# print("Inside URLs:", inside_urls)
print("Failed Fetch:", failed_fetch)
print("Splits:", len(inside_urls))
print("Tree size:", total_size)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 65/65 [00:55<00:00,  1.18it/s]

Failed Fetch: 0
Splits: 4
Tree size: 183


## Joining sub urls into one single list

In [5]:
website_urls = [item for sublist in list(inside_urls.values()) for item in sublist]
print(len(website_urls))


183


183

## Testing Date Fetching

Need to integrate Mongo 

In [8]:
fetch_date_from_url(website_urls[3])

('https://www.khaleejtimes.com/long-reads/from-instagrammer-to-war-correspondent-how-gaza-crisis-changed-this-palestinians-life',
 '15-12-2023')

## Creating {url : html content} dictionary

In [ ]:
url_html_extracted = get_html(website_urls)
url_html_extracted

## Keyword extraction performed on above dictionary

In [ ]:
url_extracted_html = kep(website_content = url_html_extracted[0], keywords = keywords, filter_by_amount = 60)

url_extracted_html

## Converting dictionary to list of tuple pairs, for implementation of batches

In [ ]:
content_list = [(key,value[:2000]) for key, value in url_extracted_html.items()] # 1000 is temporary until tokenier function is not set up
content_list

## Cutting the above list fo batches of batch size MAX_CONTENT

In [51]:
MAX_CONTENT = 6

content_list_complete = []

iterations = len(content_list) // MAX_CONTENT

# iterations = 10       ## For testing

for i in range(iterations):
    sub_content_list = content_list[MAX_CONTENT * i: MAX_CONTENT * (i + 1)]
    content_list_complete.append(sub_content_list)

# Handle remaining elements after the loop
remaining_elements = content_list[MAX_CONTENT * iterations:]
if remaining_elements:
    iterations += 1
    content_list_complete.append(remaining_elements)

In [52]:
content_list_complete = content_list_complete[:8]
len(content_list_complete)

8

## Openai's api

In [53]:
question = "Summary of situation in gaza"

response_complete = ''
for data_idx in range(0,len(content_list_complete),3):

    prompt = f""" 
        Data is in the form of tuples inside list: {content_list_complete[data_idx]} \n\n\n 
        Question: {question} \n\n\n
        Method of reply: 100 - 200 word sentences, clear reply,
        provide url if neccessary.
        """


    response = get_completion(prompt)
    response_complete += response + "\n\n"
    print(f"Batch {data_idx + 1} out of {iterations} completed ")
    

Batch 1 out of 20 completed 
Batch 4 out of 20 completed 
Batch 7 out of 20 completed 


In [54]:
response2 = get_completion(response_complete)
response2

'The situation in Gaza remains dire as the Israel-Hamas war continues. Life-saving aid for Palestinians is piling up at the border, causing delays in the delivery of essential supplies. Trucks from organizations such as UNICEF, Red Cross, and World Health Organization are stranded at the border crossing at Rafah, the only point of entry for relief. The Palestinian people are experiencing unprecedented levels of hunger, with some resorting to taking food directly from trucks out of desperation.\n\nThe war has also had a profound impact on the lives of journalists in Gaza. Plestia Alaqad, a 22-year-old Palestinian journalist, had once aimed to showcase the beauty of Gaza through her Instagram posts but found herself reporting on the war instead. The trauma of witnessing the conflict firsthand has left a lasting mark on her.\n\nAmidst the crisis, the UAE has been delivering 1.2 million gallons of water daily to Gaza through its desalination mission, providing some relief to the suffering 

In [55]:
with open("Output_gaza.txt" , "w") as f:
    f.write(response)

In [56]:
with open("Output_gaza_summary.txt" , "w") as f:
    f.write(response2)